In [1]:
import requests, zipfile, io
import pandas as pd
import os
import glob
from datetime import datetime
import shutil

In [2]:
symbol = 'BNBBUSD'
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

for month in months:
    url = f'https://data.binance.vision/data/spot/monthly/klines/{symbol}/5m/{symbol}-5m-2021-{month}.zip'
    response = requests.get(url)
    zippedFile = zipfile.ZipFile(io.BytesIO(response.content))
    zippedFile.extractall("./temp")


In [3]:
csv_files = glob.glob(os.path.join(os.getcwd() + '/temp', "*.csv"))

In [4]:
dataframe = pd.DataFrame(data = {
    'open time': [],
    'open': [],
    'high': [],
    'low': [],
    'close': [],
})

for f in csv_files:
    new_df = pd.read_csv(f)
    new_df.columns = [
        'open time',
        'open',
        'high',
        'low',
        'close',
        'Volume',
        'Close time',
        'Quote asset volume',
        'Number of trades',
        'Taker buy base asset volume',
        'Taker buy quote asset volume',
        'Ignore.'
    ]

    new_df['open time'] = new_df['open time'].apply(lambda time: datetime.utcfromtimestamp(time / 1000))
    new_df['open'] = new_df['open'].apply(lambda value: float(value))
    new_df['high'] = new_df['high'].apply(lambda value: float(value))
    new_df['low'] = new_df['low'].apply(lambda value: float(value))
    new_df['close'] = new_df['close'].apply(lambda value: float(value))
    
    new_df.drop([
        'Volume',
        'Close time',
        'Quote asset volume',
        'Number of trades',
        'Taker buy base asset volume',
        'Taker buy quote asset volume',
        'Ignore.'
    ], axis=1, inplace=True)
    
    dataframe = dataframe.append(new_df, ignore_index=True)

In [5]:
dataframe = dataframe.sort_values(by='open time')

In [6]:
dataframe.to_csv(f"{symbol}-5m.csv", index=False)

In [7]:
shutil.rmtree(os.getcwd() + '/temp')